## Create a Token on the Algorand Blockchain
#### 04.3 Winter School on Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2021-11-28

* Create an ASA token
* Transfer an ASA token
* Information about token holdings

### Setup
See notebook 04.1, the lines below will always automatically load functions in `algo_util.py`, the five accounts and the credentials

In [1]:
# Loading shared code and credentials
import sys, os
codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 5 main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [2]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.transaction import PaymentTxn
from algosdk.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
import algosdk.error
import json

In [3]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['api_token'])
algod_client.status()['last-round']

39025046

## Create an ASA = Algorand Standard Asset

This is our first token. The account MyAlgo will create an asset called `WSC Coin`. 

### Step 1: Prepare
* Remember that tokens are divisible
* We choose 2 decimals
    * Our token can be divided into units as small as $\frac{1}{10^2} = \frac{1}{100} = 0.01$
* To create 1000 tokens, we must create $1000 \cdot 10^2$ *small units*

In [4]:
sp = algod_client.suggested_params()
token_supply = 1000                              # Token supply (big units)
token_decimals =  2                              # Digits after the comma
token_total = token_supply * 10**token_decimals  # Specify number of SMALLER unit ("cents")
print(token_total)

token_name  = "Peters WSC coin"                  # <----- YOUR NAME HERE
token_url   = "www.usi.ch/wsc"                   # <----- CHANGE if you want to
token_unit  = "WSC"                              # <----- Add one initial, Abbreviation is shown in Wallet app  

100000


### Step 2: Asset creation transaction
New type of transaction, the `AssetConfigTxn`

During creation, we need to assign the following roles, which we will use later ...

* **Manager:** can change the the follwoing roles and can destroy the asset
* **Reserve:** where not yet distributed assets reside (informational)
* **Freeze:** can freeze assets (e.g. to wait for KYC)
* **Clawback:** can undo transactions

To *credibly* define that these roles are not assumed, set them to zero, using `algosdk.constants.ZERO_ADDRESS`

See https://developer.algorand.org/docs/features/asa/ 

In [5]:
txn = AssetConfigTxn(
    sender=MyAlgo['public'],                   # Creator of the ASA
    sp=sp,                      
    total=token_total,                         # Total supply in SMALL unit
    decimals=token_decimals,                   # Decimals
    default_frozen=False,                      # Are tokens frozen by default?
    unit_name=token_unit,                      # Abbreviation     
    asset_name=token_name,                     # Name
    url=token_url,                             # URL
    manager=Alice['public'],                   # Special roles (later more)
    reserve=Alice['public'],                   # Special roles
    freeze=Bob['public'],                      # Special roles
    clawback=Bob['public']                     # Special roles
)
print(txn)

{'sender': 'WSC24MVUSQ32IZYD7FNN54Z44IXWL4X7BOJD6AGFOCHOG4PDFESLZUGLTI', 'fee': 1000, 'first_valid_round': 39025047, 'last_valid_round': 39026047, 'note': None, 'genesis_id': 'testnet-v1.0', 'genesis_hash': 'SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=', 'group': None, 'lease': None, 'type': 'acfg', 'rekey_to': None, 'index': 0, 'total': 100000, 'default_frozen': False, 'unit_name': 'WSC', 'asset_name': 'Peters WSC coin', 'manager': 'ALICEXOA4Q2OD5CKBYND4UX75K3TAODEC3XCVNQ3URMKUMZKUOTOSAQLIU', 'reserve': 'ALICEXOA4Q2OD5CKBYND4UX75K3TAODEC3XCVNQ3URMKUMZKUOTOSAQLIU', 'freeze': 'BOB23JBQLW3AJREEG3KD7ULMAGYGVZ2LVF2OMZ2XBJIR64NO5P24XN7WEU', 'clawback': 'BOB23JBQLW3AJREEG3KD7ULMAGYGVZ2LVF2OMZ2XBJIR64NO5P24XN7WEU', 'url': 'www.usi.ch/wsc', 'metadata_hash': None, 'decimals': 2}


### Step 3: Sign and send

In [6]:
stxn = txn.sign(MyAlgo['private'])             # Sign
txid = algod_client.send_transaction(stxn)     # Send
print(txid)

2BYDGHRCKZTGBT63U2WUI2UM6NVFRZUPULEFXO5IJKGLJWZ4HT2A


### Step 4: Wait for confirmation

In [7]:
# Wait for the transaction to be confirmed
txinfo = wait_for_confirmation(algod_client,txid)

Current round is  39025049.
Waiting for round 39025049 to finish.
Transaction 2BYDGHRCKZTGBT63U2WUI2UM6NVFRZUPULEFXO5IJKGLJWZ4HT2A confirmed in round 39025050.


### Step 5a: Asset index and information
The asset's index is automatically created when the transaction is processed

In [8]:
print(txinfo)

{'asset-index': 642000039, 'confirmed-round': 39025050, 'pool-error': '', 'txn': {'sig': 'NC+pGS90BniJ5tgdqpNiSGk/2NGtnRzp+ExL8smK5o38YU99+Q53h6+F51uivDfMRh/LQkHZ0Gwx6kBJv+ohBg==', 'txn': {'apar': {'an': 'Peters WSC coin', 'au': 'www.usi.ch/wsc', 'c': 'BOB23JBQLW3AJREEG3KD7ULMAGYGVZ2LVF2OMZ2XBJIR64NO5P24XN7WEU', 'dc': 2, 'f': 'BOB23JBQLW3AJREEG3KD7ULMAGYGVZ2LVF2OMZ2XBJIR64NO5P24XN7WEU', 'm': 'ALICEXOA4Q2OD5CKBYND4UX75K3TAODEC3XCVNQ3URMKUMZKUOTOSAQLIU', 'r': 'ALICEXOA4Q2OD5CKBYND4UX75K3TAODEC3XCVNQ3URMKUMZKUOTOSAQLIU', 't': 100000, 'un': 'WSC'}, 'fee': 1000, 'fv': 39025047, 'gen': 'testnet-v1.0', 'gh': 'SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=', 'lv': 39026047, 'snd': 'WSC24MVUSQ32IZYD7FNN54Z44IXWL4X7BOJD6AGFOCHOG4PDFESLZUGLTI', 'type': 'acfg'}}}


In [9]:
# Get the asset ID and open in Pera Explorer
WSC_id = txinfo['asset-index']
print(WSC_id)
print(cred['explore_test']+'asset/{}'.format(WSC_id))

642000039
https://testnet.explorer.perawallet.app/asset/642000039


#### Step 5b: Asset holdings in the wallet app
**CHECK** your wallet app, where the holdings of WSC will already have shown up.

In [10]:
# We can also check the assets that we have created
account_info = algod_client.account_info(MyAlgo['public'])
json_str = json.dumps(account_info['created-assets'])
json.loads(json_str)

[{'index': 592359981,
  'params': {'clawback': 'BOB23JBQLW3AJREEG3KD7ULMAGYGVZ2LVF2OMZ2XBJIR64NO5P24XN7WEU',
   'creator': 'WSC24MVUSQ32IZYD7FNN54Z44IXWL4X7BOJD6AGFOCHOG4PDFESLZUGLTI',
   'decimals': 2,
   'default-frozen': False,
   'freeze': 'BOB23JBQLW3AJREEG3KD7ULMAGYGVZ2LVF2OMZ2XBJIR64NO5P24XN7WEU',
   'manager': 'ALICEXOA4Q2OD5CKBYND4UX75K3TAODEC3XCVNQ3URMKUMZKUOTOSAQLIU',
   'name': 'Peters WSC coin',
   'name-b64': 'UGV0ZXJzIFdTQyBjb2lu',
   'reserve': 'ALICEXOA4Q2OD5CKBYND4UX75K3TAODEC3XCVNQ3URMKUMZKUOTOSAQLIU',
   'total': 100000,
   'unit-name': 'PWSC',
   'unit-name-b64': 'UFdTQw==',
   'url': 'www.usi.ch/wsc',
   'url-b64': 'd3d3LnVzaS5jaC93c2M='}},
 {'index': 592373389,
  'params': {'clawback': 'WSC24MVUSQ32IZYD7FNN54Z44IXWL4X7BOJD6AGFOCHOG4PDFESLZUGLTI',
   'creator': 'WSC24MVUSQ32IZYD7FNN54Z44IXWL4X7BOJD6AGFOCHOG4PDFESLZUGLTI',
   'decimals': 0,
   'default-frozen': False,
   'freeze': 'WSC24MVUSQ32IZYD7FNN54Z44IXWL4X7BOJD6AGFOCHOG4PDFESLZUGLTI',
   'manager': 'WSC24MVU

## Transfer coins
In this section we will transfer coins ...
* Manually to Alice
* Using Python to Bob

### Manual transfer to Alice
* Alice has to opt-in
    * Tap on `Alice` > Tap on green `+` next to "Assets" > Find name or asset ID on list > Tap on `+` >  Approve transaction
* Now you can send 10 WSC via QR code or copy-paste
* To send **your** WSC to your neigbhor, she has to opt into **your** WSC coin!

### Exercise: Python transfer to Bob 
* `MyAlgo` transfers 10 WSC coins to `Bob`
* Remember that transfers of tokens (ASA) are performed with `AssetTransferTxn()` and not with `PaymentTxn()`
* Also, do not forget the ...

In [11]:
# Your Python code goes here ...

# Step 1: prepare AssetTransferTxn
sp = algod_client.suggested_params()             # suggested params
amt = int(0)                                     # <-- send 0 coins

txn = AssetTransferTxn(
    sender = Bob['public'],                    # <-- Alice sends ...
    sp=sp,
    receiver=Bob['public'],                    # <-- ... to herself
    amt=amt,
    index=WSC_id                               # <-- specify ASA = WSC coin
    )                               

# Step 2+3: sign and send
stxn = txn.sign(Bob['private'])               # Sign
txid = algod_client.send_transaction(stxn)      # Send

# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client,txid)


Current round is  39025054.
Waiting for round 39025054 to finish.
Waiting for round 39025055 to finish.
Transaction 6QKBOY552LDDYVQVFB6LWDDGXRJQTT44MJN6PMHWY6GCVXMZB3OQ confirmed in round 39025056.


In [12]:
# More Python code may go here ...

# Step 1: prepare AssetTransferTxn
sp = algod_client.suggested_params()             # suggested params
amt = int(10 * 1e2)                              # <-- send 10 coins

txn = AssetTransferTxn(
    sender = MyAlgo['public'],                    # <-- Alice sends ...
    sp=sp,
    receiver=Bob['public'],                     # <-- ... to herself
    amt=amt,
    index=WSC_id                                # <-- specify ASA
    )                               

# Step 2+3: sign and send
stxn = txn.sign(MyAlgo['private'])               # Sign
txid = algod_client.send_transaction(stxn)      # Send

# Step 4: Wait for confirmation
txinfo = wait_for_confirmation(algod_client,txid)

Current round is  39025056.
Waiting for round 39025056 to finish.
Waiting for round 39025057 to finish.
Transaction OADRNECL2FZTMUD3ZIO72G56IDHRDJUO3QADI5KSWAILYULNBLIA confirmed in round 39025058.


## Appendix: Get (more/better) information
* Better overview of **all** holdings of one account
* `asset_holdings_df` that produces a nice list of all the asset holdings of an address
* It is part of `algo_util.py` 

In [13]:
!pip install pandas

In [14]:
asset_holdings_df(algod_client,Bob['public'])

,amount,unit,asset-id,name,decimals
0,22.578488,ALGO,0,Algorand,6
1,2.097569,USDC,10458941,USDC,6
2,8.100000,PWSC,592359981,Peters WSC coin,2
3,25.000000,TEMP,641963975,Peters Tempcoin,1
4,10.000000,WSC,642000039,Peters WSC coin,2


### Account info
* Information about an **account**
* Obtain ALGOs with `'amount'`
* Obtain all other tokens with `'assets'`

In [15]:
account_info = algod_client.account_info(Bob['public'])
account_info

{'address': 'BOB23JBQLW3AJREEG3KD7ULMAGYGVZ2LVF2OMZ2XBJIR64NO5P24XN7WEU',
 'amount': 22578488,
 'amount-without-pending-rewards': 22578488,
 'apps-local-state': [{'id': 592631026,
   'schema': {'num-byte-slice': 0, 'num-uint': 0}},
  {'id': 592636761,
   'key-value': [{'key': 'Tm90ZQ==',
     'value': {'bytes': 'V2VsY29tZSBvbiBib2FyZCE=', 'type': 1, 'uint': 0}}],
   'schema': {'num-byte-slice': 1, 'num-uint': 0}},
  {'id': 592637438,
   'key-value': [{'key': 'TWVtYmVyc2hpcE5v',
     'value': {'bytes': '', 'type': 2, 'uint': 3}}],
   'schema': {'num-byte-slice': 0, 'num-uint': 1}},
  {'id': 592638886,
   'key-value': [{'key': 'TWVtYmVyc2hpcE5v',
     'value': {'bytes': '', 'type': 2, 'uint': 3}}],
   'schema': {'num-byte-slice': 0, 'num-uint': 1}},
  {'id': 592753444,
   'key-value': [{'key': 'TGFzdEhvbGRpbmc=',
     'value': {'bytes': '', 'type': 2, 'uint': 19085945}},
    {'key': 'TWVzc2FnZQ==',
     'value': {'bytes': 'WW91IGhhdmUgZW5vdWdo', 'type': 1, 'uint': 0}},
    {'key': 'Vmlza

In [16]:
print(account_info['amount'])         # micro Algos
print(account_info['amount']/1E6)     # Algos

22578488
22.578488


In [17]:
account_info['assets']                # All ASA

[{'amount': 2097569, 'asset-id': 10458941, 'is-frozen': False},
 {'amount': 810, 'asset-id': 592359981, 'is-frozen': False},
 {'amount': 250, 'asset-id': 641963975, 'is-frozen': True},
 {'amount': 1000, 'asset-id': 642000039, 'is-frozen': False}]

### Asset Info
* Information about a specific **asset**
* Specially useful is the `'decimals'`

In [18]:
asset_info = algod_client.asset_info(WSC_id)
asset_info

{'index': 642000039,
 'params': {'clawback': 'BOB23JBQLW3AJREEG3KD7ULMAGYGVZ2LVF2OMZ2XBJIR64NO5P24XN7WEU',
  'creator': 'WSC24MVUSQ32IZYD7FNN54Z44IXWL4X7BOJD6AGFOCHOG4PDFESLZUGLTI',
  'decimals': 2,
  'default-frozen': False,
  'freeze': 'BOB23JBQLW3AJREEG3KD7ULMAGYGVZ2LVF2OMZ2XBJIR64NO5P24XN7WEU',
  'manager': 'ALICEXOA4Q2OD5CKBYND4UX75K3TAODEC3XCVNQ3URMKUMZKUOTOSAQLIU',
  'name': 'Peters WSC coin',
  'name-b64': 'UGV0ZXJzIFdTQyBjb2lu',
  'reserve': 'ALICEXOA4Q2OD5CKBYND4UX75K3TAODEC3XCVNQ3URMKUMZKUOTOSAQLIU',
  'total': 100000,
  'unit-name': 'WSC',
  'unit-name-b64': 'V1ND',
  'url': 'www.usi.ch/wsc',
  'url-b64': 'd3d3LnVzaS5jaC93c2M='}}

In [19]:
# Useful fields
print(asset_info['params']['decimals'])
print(asset_info['params']['unit-name'])

2
WSC


### Putting it together
* Crate a Python list with all relevant holdings of an account

In [20]:
info = []
# Algo part
info.append( {'amount':  account_info['amount']/1E6, 'unit' :   'ALGO', 'asset-id': 0, 'name': 'Algorand' } )
# ASA part
assets = account_info['assets']
for asset in assets:
    asset_id     = asset['asset-id']
    asset_info   = algod_client.asset_info(asset_id)                         # Get all info
    asset_amount = asset['amount']/10**asset_info['params']['decimals']      # convert to BIG units
    info.append( {'amount':  asset_amount,
                  'unit' :   asset_info['params'].get('unit-name'),
                  'asset-id':asset_id,
                  'name':    asset_info['params'].get('name')
                  } )

info

[{'amount': 22.578488, 'unit': 'ALGO', 'asset-id': 0, 'name': 'Algorand'},
 {'amount': 2.097569, 'unit': 'USDC', 'asset-id': 10458941, 'name': 'USDC'},
 {'amount': 8.1,
  'unit': 'PWSC',
  'asset-id': 592359981,
  'name': 'Peters WSC coin'},
 {'amount': 25.0,
  'unit': 'TEMP',
  'asset-id': 641963975,
  'name': 'Peters Tempcoin'},
 {'amount': 10.0,
  'unit': 'WSC',
  'asset-id': 642000039,
  'name': 'Peters WSC coin'}]

### Packaging as a function and creating a data.frame
* `asset_holdings()` returns a list
* `asset_holdings_df()` returns a pandas data frame

Both functions are part of `algo_util.py`.

In [21]:
def asset_holdings(algod_client, public):
    # client = algosdk client
    # public = public address to be analyzed

    import pandas as pd
    from algosdk.v2client import algod
    account_info = algod_client.account_info(public)

    info = []
    # Algo part
    info.append( {'amount':  account_info['amount']/1E6, 
                  'unit' :   'ALGO', 'asset-id': 0, 'name': 'Algorand'} )

    # ASA part
    assets = account_info['assets']
    for asset in assets:
        asset_id     = asset['asset-id']
        asset_info   = algod_client.asset_info(asset_id)                         # Get all info
        asset_amount = asset['amount']/10**asset_info['params']['decimals']      # convert to BIG units
        info.append( {'amount':  asset_amount,
                      'unit' :   asset_info['params'].get('unit-name'),
                      'asset-id':asset_id,
                      'name':    asset_info['params'].get('name')
                      } )

    return(info)

In [22]:
asset_holdings(algod_client,Alice['public'])

[{'amount': 25.122489, 'unit': 'ALGO', 'asset-id': 0, 'name': 'Algorand'},
 {'amount': 279.440275, 'unit': 'USDC', 'asset-id': 10458941, 'name': 'USDC'},
 {'amount': 2.5,
  'unit': 'PWSC',
  'asset-id': 592359981,
  'name': 'Peters WSC coin'},
 {'amount': 0.0,
  'unit': 'USDALGO',
  'asset-id': 592647685,
  'name': 'USD/Algo Oracle coin'},
 {'amount': 0.0,
  'unit': 'WSC',
  'asset-id': 641958950,
  'name': 'Peters WSC coin'}]

In [23]:
def asset_holdings_df(algod_client, public):
    import pandas as pd
    from algosdk.v2client import algod
    info = asset_holdings(algod_client, public)
    df = pd.DataFrame(info)
    return(df)

In [24]:
asset_holdings_df(algod_client,Alice['public'])

,amount,unit,asset-id,name
0,25.122489,ALGO,0,Algorand
1,279.440275,USDC,10458941,USDC
2,2.500000,PWSC,592359981,Peters WSC coin
3,0.000000,USDALGO,592647685,USD/Algo Oracle coin
4,0.000000,WSC,641958950,Peters WSC coin


### *Python note on `.get()`
* Some entries in `asset_info['params']` are optional
    * So they may not exist
* Using the `.get()` method avoids error messages

In [25]:
# Example of a minimal ASA
asset_info = {'index': 145346757, 
                'params': {'creator': 'CPUT3Z5CI3XOIZ4ARSGUFQD7V4YGYJW5BFAZMXX5YOV4KJCKI6MBCDY5XM', 
                           'decimals': 0, 'default-frozen': False, 
                           'name': 'TEMP', 'name-b64': 'VEVNUA==', 'total': 100}}

In [26]:
print(asset_info['params']['name'])
# this does not work
print(asset_info['params']['unit-name'])

TEMP


KeyError: 'unit-name'

In [27]:
# this works
print(asset_info['params'].get('name'))
print(asset_info['params'].get('unit-name'))

TEMP
None
